In [7]:
import requests, time
from bs4 import BeautifulSoup

def get_news_list(root, url, date_boundary):
    article_list = []
    while 1:
        path = root + url
        html = requests.get(path)
        soup = BeautifulSoup(html.text)
        body = soup.find('ul', 'wp_article_list')  # 寻找对应的新闻列表
        for item in body.find_all('li'):  # 遍历新闻列表中的每条新闻组
            date = item.find('span', 'Article_PublishDate').string  # 通过每条新闻组中 	span的class获取时间字符串（不用.string获取的为tag）
            convert_date = time.mktime(time.strptime(date, "%Y-%m-%d"))  # 用秒数来表示时间
            if convert_date < date_boundary:
                break
            title = item.find('a')['title']
            href = item.find('a')['href']
            article_list.append([date, title, href])  # 将数据追加保存到article_list中
        # print(title, href ,date)
        if time.mktime(time.strptime(date, '%Y-%m-%d')) > date_boundary:
            next_page = soup.find('ul', 'wp_paging clearfix')
            url = next_page.find('a', {'class': 'next'})['href']
        else:
            break
    #print(article_list)
    return article_list

In [8]:
def crawl_news(root,article_list):
    news_dict = {}
    i = 1
    for item in article_list:
        date,title,href = item # 将数组中的三个元素分别赋值给变量
        #print(date,title,href)
        if 'http' in href:
            path = href
        else:
            path = root + href
        f = requests.get(path)
        soup = BeautifulSoup(f.text)
        body = soup.find('div',{'class','acd'})
        content = body.find('div',{'class','wp_articlecontent'})
        text = u''
        for a in content.strings: # 效果等同于text = content.text
            text += a
        
        publis = body.find_all('span','arti_publisher')[1].string
        department = publis.split(u'\uff1a')[1] # 将publis通过 ':'进行分割，效果等同于 publis.split(':')
        if department == '':
            department = 'none'
        view_times = body.find('span',['class','WP_VisitCount']).text
        #逐个把新闻信息加入字典
        news_dict[i] = {'date':date,'title':title,'source':department,'content':text,'views':view_times,'url':path}
        
        time.sleep(0.1) # 控制爬虫速度
        i += 1;
    return news_dict

def search(keywords,news_dict):
    result = {} # result为字典数据类型
    title_list = [] # 列表数据类型
    source_list = []
    content_list = []
    for x in news_dict: # 对于字典数据类型，x的值为1、2、3......
        if keywords in news_dict[x]['title']:
            title_list.append([x,news_dict[x]['date'],news_dict[x]['title']])
        if keywords in news_dict[x]['source']:
            source_list.append([x,news_dict[x]['date'],news_dict[x]['source']])
        if keywords in news_dict[x]['content']:
            content_list.append([x,news_dict[x]['date'],news_dict[x]['content']])
    
    result['title'] = title_list
    result['source'] = source_list
    result['content_list'] = content_list
    return result
        # print(x)

In [9]:
if __name__ == '__main__': # 如果这个module文件是被当成程序来执行，那么，该__name__属性的值就是"__main__"
    root = "http://www.suibe.edu.cn/_s19"
    url = "/1416/list.psp"
    date_boundary = time.mktime(time.strptime("2019-08-01", "%Y-%m-%d"))
    news_list = get_news_list(root, url, date_boundary)
    news_dict = crawl_news(root,news_list)
    print(news_dict)

{1: {'date': '2019-10-01', 'title': '学校隆重举行少数民族联合会成立大会', 'source': '党委统战部', 'content': '9月30日下午，学校少数民族联合会成立大会暨一届一次会员大会在信息楼503会议室隆重举行。上海对外经贸大学党委副书记吴毅、上海市少数民族联合会副秘书长杨翼出席大会，东华大学少数民族联合会会长张瑞云、上海对外经贸大学民建支部主委郭振华到会祝贺。会议由党委统战部欧阳君主持。\xa0\xa0会议审议并通过了上海对外经贸大学少数民族联合会章程，选举产生了第一届理事会成员。在一届一次理事会议上选举崔树林（朝鲜族）为会长，谷永芬（回族）、孙雪瑛（满族）为副会长，刘春华（壮族）为秘书长。杨翼对学校成立少数民族联合会表示祝贺，并对今后民族联开展好工作提出了三点希望：一是要努力学习，与时俱进，深刻领会党中央对做好新时代民族工作的精神要求，联系实际，扎实做好各项民族工作。二是要努力加压，坚持以问题为导向、以需求为导向，以目标为导向，既注重发扬传统，又要努力创新服务。三是要夯实工作基础，提升民族联自身的综合素质和能力水平，增强民族工作的感染力、凝聚力和影响力，展示上海对外经贸大学少数民族同胞的风采。\xa0\xa0吴毅代表学校党委对市民族联莅临学校指导工作，对东华大学民族联到会祝贺表示感谢，对学校民族联的成立和新当选的第一届班子成员表示祝贺。他强调，长期以来民族工作的实践充分证明，少数民族联合会是联系团结少数民族群众的重要载体，是少数民族群众发挥作用的重要平台，是维护民族团结和社会稳定的重要力量。在新中国成立70周年之际，在“不忘初心、牢记使命”主题教育期间，学校成立民族联具有特殊的重要意义！特别是9月27日，习近平总书记在全国民族团结进步表彰大会上强调，实现中华民族伟大复兴，需要各民族手挽着手、肩并着肩，共同努力奋斗。\xa0\xa0他指出，当前学校发展正处于“爬坡过坎”的关键时期，需要广大统一战线成员以兴校强校的使命担当和奋斗精神，主动参与学校的各项中心工作，把主题教育中激发出的爱国热情转化为立足岗位、发奋工作的实际行动，转化为攻坚克难、干事创业的实际成效，为推动学校高质量、内涵式发展发挥更加重要的作用。他希望民族联始终以习近平新时代中国特色社会主义思想为指引，不忘初心、牢记使命，在学校党委的坚强领导下，切实加

In [10]:
result = search(u'上海对外经贸大学专场',news_dict)
for x in result['title']:
    print(x[0],x[1])
    print(x[2])